<a href="https://colab.research.google.com/github/angela24680403/LLM_Mathematical_Reasoning_Experiments/blob/main/Prompt_Template_Experiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth vllm
else:
    # [NOTE] Do the below ONLY in Colab! Use [[pip install unsloth vllm]]
    !pip install --no-deps unsloth vllm
    # ==0.7.3

In [2]:
#@title Colab Extra Install { display-mode: "form" }
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth vllm
else:
    !pip install --no-deps unsloth vllm==0.7.3
    # [NOTE] Do the below ONLY in Colab! Use [[pip install unsloth vllm]]
    # Skip restarting message in Colab
    import sys, re, requests; modules = list(sys.modules.keys())
    for x in modules: sys.modules.pop(x) if "PIL" in x or "google" in x else None
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft "trl==0.15.2" triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf datasets huggingface_hub hf_transfer

    # vLLM requirements - vLLM breaks Colab due to reinstalling numpy
    f = requests.get("https://raw.githubusercontent.com/vllm-project/vllm/refs/heads/main/requirements/common.txt").content
    with open("vllm_requirements.txt", "wb") as file:
        file.write(re.sub(rb"(transformers|numpy|xformers)[^\n]{1,}\n", b"", f))
    !pip install -r vllm_requirements.txt

In [21]:
from unsloth import FastLanguageModel, is_bfloat16_supported
import torch
max_seq_length = 16384 # Can increase for longer reasoning traces
lora_rank = 64 # Larger rank = smarter, but slower

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/DeepSeek-R1-Distill-Qwen-1.5B",
    max_seq_length = max_seq_length,
    load_in_4bit = True, # False for LoRA 16bit
    fast_inference = True, # Enable vLLM fast inference
    max_lora_rank = lora_rank,
    gpu_memory_utilization = 0.5,# Reduce if out of memory
)

Unsloth: Switching from Unsloth dynamic quant to normal quant since
we do not yet support fast inference for unsloth/deepseek-r1-distill-qwen-1.5b-unsloth-bnb-4bit
==((====))==  Unsloth 2025.5.9: Fast Qwen2 patching. Transformers: 4.52.3. vLLM: 0.7.3.
   \\   /|    NVIDIA L4. Num GPUs = 1. Max memory: 22.161 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.9. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: vLLM loading unsloth/deepseek-r1-distill-qwen-1.5b-bnb-4bit with actual GPU utilization = 16.2%
Unsloth: Your GPU has CUDA compute capability 8.9 with VRAM = 22.16 GB.
Unsloth: Using conservativeness = 1.0. Chunked prefill tokens = 16384. Num Sequences = 128.
Unsloth: vLLM's KV Cache can use up to 1.82 GB. Also swap space = 4 GB.
INFO 06-06 09:20:09 config

Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


INFO 06-06 09:20:15 model_runner.py:1115] Loading model weights took 1.5052 GB
INFO 06-06 09:20:18 worker.py:267] Memory profiling takes 2.23 seconds
INFO 06-06 09:20:18 worker.py:267] the current vLLM instance can use total_gpu_memory (22.16GiB) x gpu_memory_utilization (0.16) = 3.59GiB
INFO 06-06 09:20:18 worker.py:267] model weights take 1.51GiB; non_torch_memory takes 0.00GiB; PyTorch activation peak memory takes 1.08GiB; the rest of the memory reserved for KV Cache is 1.00GiB.
INFO 06-06 09:20:19 executor_base.py:111] # cuda blocks: 2348, # CPU blocks: 9362
INFO 06-06 09:20:19 executor_base.py:116] Maximum concurrency for 16384 tokens per request: 2.29x
INFO 06-06 09:20:19 model_runner.py:1434] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI. If out-of-memory error occurs during cudagraph capture, consider decreasing `gpu_memory_utili


Capturing CUDA graph shapes: 100%|██████████| 19/19 [00:30<00:00,  1.58s/it]

INFO 06-06 09:20:49 model_runner.py:1562] Graph capturing finished in 30 secs, took 0.06 GiB
INFO 06-06 09:20:49 llm_engine.py:436] init engine (profile, create kv cache, warmup model) took 33.47 seconds


Unsloth: Just some info: will skip parsing ['q_norm', 'pre_feedforward_layernorm', 'k_norm', 'post_feedforward_layernorm']
Unsloth: Just some info: will skip parsing ['q_norm', 'pre_feedforward_layernorm', 'k_norm', 'post_feedforward_layernorm']


In [15]:
import re

def extract_boxed_value(text):
    # Use regular expression to find the value inside \boxed{}
    pattern = r'\\boxed{([^}]*)}'
    match = re.search(pattern, text)

    if match:
        return match.group(1)
    else:
        return None

In [5]:
!pip install -U datasets

In [16]:
from datasets import load_dataset, Dataset

In [17]:
try:
    aime24 = load_dataset("simplescaling/aime24_nofigures", revision="main")['train']
    print("Dataset loaded successfully with revision 'main'")
except Exception as e:
    print(f"Loading with revision 'main' failed: {e}")

Dataset loaded successfully with revision 'main'


In [18]:
from vllm import SamplingParams
sampling_params = SamplingParams(
    temperature = 0.0,
    max_tokens = max_seq_length,
)

In [23]:
SYS_PROMPT1 = """
Solve the following math problem efficiently and clearly.
The last line of your response should be of the following
format: ’Therefore, the final answer is: $\boxed{ANSWER}$.
I hope it is correct’ (withoutquotes) where ANSWER is just
the final number or expression that solves the problem.
Think step by step before answering.\n
"""

SYS_PROMPT2 = """
Act as a highly intelligent student who is very good at answering exam math questions.
Your audience is a teacher who is marking the exam, looking for the correct working out and answer to the maths problem.
The last line of your response should be of the following
format: ’Therefore, the final answer is: $\boxed{ANSWER}$.
I hope it is correct’. Be logical concise in the reasoning process.
"""

SYS_PROMPT3 = """
Act as a highly intelligent student who is very good at answering exam math questions.
Your audience is a teacher who is marking the exam, looking for the correct working out and answer to the maths problem.
Be logical concise in the reasoning process.
Please extract the question-solving information related to the problem. Only extract the most useful information, list them one by one!
Please understand the information extracted from the maths problem,then solve the question step by step and
show the answer.
The last line of your response should be of the following
format: ’Therefore, the final answer is: $\boxed{ANSWER}$.
I hope it is correct’.
"""

sys_prompts = [SYS_PROMPT1, SYS_PROMPT2, SYS_PROMPT3]

In [ ]:

pretrained_outputs = []
answers = []
example = aime24[5]
for sys_prompt in sys_prompts:
    text = tokenizer.apply_chat_template([
        {'role': 'system', 'content': sys_prompt},
        {"role" : "user", "content" : example['problem']},
    ], tokenize = False, add_generation_prompt = True)

    output = model.fast_generate(
      [text],
      sampling_params = sampling_params,
      lora_request = None,
      )[0].outputs[0].text
    pretrained_outputs.append(output)
    answers.append(extract_boxed_value(output))

print("The correct answer: " + example['answer'])
print("Generated answers: " + str(answers))



Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]